# Tutorial: Embedded Bethe-Salpeter Equation (eBSE) Calculations with WEST

In this tutorial, we show how to perform embedded BSE (eBSE) calculations with the WEST code. Within eBSE, we formulate an effective Hamiltonian for a selected number of orbitals in the form
$$
H^{\mathrm{eBSE}}_{vc, v'c'} = \left( \epsilon^{GW}_{c} - \epsilon^{GW}_{v} \right)\delta_{vv'}\delta_{cc'} - W_{cv, c'v'} + V_{cv, c'v'},
$$
where $v$ and $v'$ include all occupied states in the active space, $c$ and $c'$ all unoccupied states. $\epsilon^{GW}$ are the quasiparticle energies in the $\mathrm{G_0W_0}$ approximation.

Matrix elements of the direct interaction $W_{cv, c'v'}$ are defined as
$$
w_{cv,c'v'} = \int d^3\mathbf{r} d^3\mathbf{r}' \phi_{v}(\mathbf{r})\phi_{v'}(\mathbf{r})W_0(\mathbf{r},\mathbf{r}',\omega=0) \phi_{c}(\mathbf{r}')\phi_{c'}(\mathbf{r}'),
$$
where $W_0(\mathbf{r},\mathbf{r}',\omega)$ is the screened Coulomb potential in the random-phase approximation (RPA). These matrix elements describe the Coulomb attraction between the excited electrons and the valence hole.

Matrix elements of the repulsive exchange interaction $V_{cv, c'v'}$ are given by
$$
V_{cv,c'v'} = \int d^3\mathbf{r} d^3\mathbf{r}'  \phi_{v}(\mathbf{r})\phi_{c}(\mathbf{r})W^R_0(\mathbf{r},\mathbf{r}',\omega=0) \phi_{v'}(\mathbf{r}')\phi_{c'}(\mathbf{r}'),
$$
where $W^R_0(\mathbf{r},\mathbf{r}',\omega)$ is the Coulomb potential in the constrained random-phase approximation (cRPA).


The eigenvalues and -states of the eBSE Hamiltonian yield the neutral excitations of the system, i.e.
$$
H^{\mathrm{eBSE}}X^\lambda = E^\lambda X^\lambda,
$$
where $E^\lambda$ are the neutral excitation energies, and $X^\lambda$ are the corresponding excited many-body states.

eBSE calculations are performed in two separate steps:
1. In an initial WEST calculation, the quasiparticle energies and matrix elements $W$ and $V$ are calculated and stored in a JSON file.
2. The eBSE Hamiltonian is constructed from the parameters in the JSON file and diagonalized. This step is performed by the WESTpy library.

# Step 1: Parameters of the eBSE Hamiltonian

## Step 1.1: Mean-Field Starting Point

As a starting point for the many-body perturbation theory (MBPT) calculation, we perform a density functional theory (DFT) calculation using [Quantum ESPRESSO](https://www.quantum-espresso.org/).

Note that embedded BSE calculations require spin-polarized DFT calculations as a starting point. This is distinct from Quantum Defect Embedding Theory (QDET), which generally starts from spin-unpolarized DFT calculations.

Download the following files in your working directory:

In [ ]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63_spinpol/pw.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.0.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/N_ONCV_PBE-1.0.upf

Let us inspect the `pw.in` file:

In [2]:
%%bash
cat pw.in

&CONTROL
calculation = 'scf'
wf_collect = .true.
pseudo_dir = './'
/
&SYSTEM
input_dft = 'PBE'
ibrav = 0
ecutwfc = 50
nosym = .true.
tot_charge = -1
nspin = 2
tot_magnetization = 2
nbnd = 176
nat = 63
ntyp = 2
/
&ELECTRONS
conv_thr = 1D-08
/
K_POINTS gamma
CELL_PARAMETERS angstrom
7.136012  0.000000  0.000000
0.000000  7.136012  0.000000
0.000000  0.000000  7.136012
ATOMIC_SPECIES
C  12.01099968  C_ONCV_PBE-1.0.upf
N  14.00699997  N_ONCV_PBE-1.0.upf
ATOMIC_POSITIONS crystal
C    0.99996000  0.99996000  0.99996000
C    0.12495000  0.12495000  0.12495000
C    0.99905000  0.25039000  0.25039000
C    0.12350000  0.37499000  0.37499000
C    0.25039000  0.99905000  0.25039000
C    0.37499000  0.12350000  0.37499000
C    0.25039000  0.25039000  0.99905000
C    0.37499000  0.37499000  0.12350000
C    0.00146000  0.00146000  0.50100000
C    0.12510000  0.12510000  0.62503000
C    0.00102000  0.24944000  0.74960000
C    0.12614000  0.37542000  0.87402000
C    0.24944000  0.00102000  0.74960000
C

We can now run `pw.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 pw.x -i pw.in > pw.out

## Step 1.2: Calculation of the Dielectric Screening

As for GW and QDET calculations within WEST, we first have to determine the dielectric screening before we can proceed to calculate the excitations of the spin defect. In WEST, the dielectric screening is obtained from the projective dielectric eigendecomposition (PDEP) technique. The calculation with `wstat.x` requires an input file `wstat.in`.

In [ ]:
%%bash 
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63_spinpol/wstat.in

Once again, we can have a look at the input file:

In [4]:
%%bash
cat wstat.in

wstat_control:
  wstat_calculation: S
  n_pdep_eigen: 512
  trev_pdep: 0.00001


As we can see, there are no input parameters in `wstat.in` specific to QDET. We can now execute `wstat.x` with the following command.

In [ ]:
%%bash
mpirun -n 2 wstat.x -i wstat.in > wstat.out

## Step 1.3: eBSE Calculation

The calculation of matrix elements for eBSE within WEST is identical to that of matrix elements for QDET, with the exception of a single keyword: By adding the keyword `l_qdet_verbose: true`, we enable the write-out of both matrix elements of the RPA-screened and cRPA-screened matrix elements (typically only matrix elements of the cRPA-screened potential are written to JSON file).

In [ ]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63_spinpol/wfreq.in

We see that the `wfreq.in` look exactly like the input file used for QDET calculations.

In [5]:
%%bash
cat wfreq.in

wstat_control:
  wstat_calculation: S
  n_pdep_eigen: 512
  trev_pdep: 0.00001

wfreq_control:
  wfreq_calculation: XWGQH
  macropol_calculation: C
  l_qdet_verbose: true
  l_enable_off_diagonal: true
  n_pdep_eigen_to_use: 512
  qp_bands: [87, 122, 123, 126, 127, 128]
  n_refreq: 300
  ecut_refreq: 2.0


The `wfreq.x` calculation is performed with the following command

In [ ]:
%%bash
mpirun -n 2 wfreq.x -i wfreq.in > wfreq.out

To avoid the computationally expensive `wstat.x` and `wfreq.x` calculations, we can also download the JSON files with the following commands.

In [ ]:
%%bash
mkdir west.wfreq.save
wget -N -q http://www.west-code.org/doc/training/nv_diamond_63_spinpol/wfreq.json -O west.wfreq.save/wfreq.json

# Step 2: Diagonalization of the eBSE Hamiltonian

In [2]:
import json
import numpy as np

from westpy.qdet import eBSEResult

# construct object for effective Hamiltonian
ebse = eBSEResult(filename='west.wfreq.save/wfreq.json', spin_flip_=True)

# diagonalize Hamiltonian
solution = ebse.solve()

 
 _    _ _____ _____ _____            
| |  | |  ___/  ___|_   _|           
| |  | | |__ \ `--.  | |_ __  _   _  
| |/\| |  __| `--. \ | | '_ \| | | | 
\  /\  / |___/\__/ / | | |_) | |_| | 
 \/  \/\____/\____/  \_/ .__/ \__, | 
                       | |     __/ | 
                       |_|    |___/  
 
WEST version     :  5.2.0
Today            :  2022-12-02 17:48:51.410643
Solving eBSE Hamiltonian...


diag[1RDM - 1RDM(GS)]                                   
   E [eV]  char                    87    122    123    126    127    128
0   0.000 2.993                 0.000  0.000  0.000  0.000  0.000  0.000
1   0.427 1.081                -0.000 -0.003 -0.003 -0.014  0.000  0.021
2   0.427 1.081                -0.000 -0.003 -0.003 -0.014  0.021 -0.000
3   1.054 1.022                -0.000 -0.000 -0.000 -0.000 -0.000  0.000
4   2.247 2.206                -0.000 -0.002 -0.006 -0.474  0.483  0.000
5   2.247 2.206                -0.000 -0.002 -0.006 -0.474  0.000  0.483
6   4.038 2.232                -0.001 -0.005 -0.483 -0.009  0.497  0.000
7   4.038 2.232                -0.001 -0.005 -0.483 -0.009  0.000  0.497
8   5.865 2.233                -0.002 -0.487 -0.007 -0.002  0.498  0.000
9   5.865 2.233                -0.002 -0.487 -0.007 -0.002  0.000  0.498
10  9.300 2.236                -0.497 -0.002 -0.000 -0.000  0.500  0.000
11  9.300 2.236                -0.497 -0.002 -0.000 -0.000  0.000  0.500

-----------------------------------------------------


The output of the eBSE diagonalization looks very similar to the output of a QDET calculation.

As in the case of QDET, we print a list of eigenvalues and their energies (first and second column). For each state, we provide the character, i.e. the spin-multiplicity in the third column. The following column describe the change in occupation relative to the groundstate.

Let's consider the 1st excitation as an example: The state is 0.427 eV above the groundstate and has a multiplicity of 1.081. That means that it is a singlet, which should have a multiplicity of 1. The difference is called the *spin contamination*, describing that the spin state is note pure.

For a more detailed analysis, one can access the full output stored in the `solution` dictionary:

In [3]:
print([key for key in solution.keys()])

['hamiltonian', 'evs_au', 'evs', 'evcs', 'rdm1s', 'mults', 'excitations']


## Comparison between eBSE and QDET 

We can also easily compare the results from eBSE and QDET as we can perform both calculations from the same output:

In [4]:
from westpy.qdet import QDETResult

# construct object for effective Hamiltonian
qdet = QDETResult(filename='west.wfreq.save/wfreq.json')

# diagonalize Hamiltonian
qdet_solution = qdet.solve(nelec=(5,5))

-----------------------------------------------------
Building effective Hamiltonian...
nspin: 2
occupations: [[1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 0.]]


diag[1RDM - 1RDM(GS)]                                 
  E [eV] char                    87    122    123    126   127   128
0  0.000   3-                 0.000  0.000  0.000  0.000 0.000 0.000
1  0.418   1-                -0.000 -0.005 -0.008 -0.035 0.014 0.035
2  0.418   1-                -0.000 -0.005 -0.008 -0.035 0.035 0.014
3  1.207   1-                -0.001 -0.010 -0.011 -0.036 0.028 0.029
4  1.835   3-                -0.001 -0.007 -0.061 -0.429 0.449 0.050
5  1.835   3-                -0.001 -0.007 -0.061 -0.429 0.050 0.449
6  2.806   1-                -0.000 -0.022 -0.017 -0.426 0.443 0.022
7  2.806   1-                -0.000 -0.022 -0.017 -0.426 0.022 0.443
8  4.472   1-                -0.003 -0.033 -0.082 -0.842 0.480 0.480
9  4.896   2-                -0.008 -0.356 -0.101 -0.034 0.475 0.024

-----------------------------------------------------


As we can see, both approaches yield similar energies and qualitatively the same level diagram (triplet groundstate followed by 3 singlet excitations), but the spin multiplicity is different. Generally, we observe more pronounced spin contamination in eBSE than in QDET.